In [1]:
from google.colab import drive

drive.mount('/content/gdrive')
!cp -r gdrive/MyDrive/translation-for-simplified-language-1/data/kurier/. .

Mounted at /content/gdrive


In [2]:
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
import unicodedata
import re

#General Stuff

In [3]:
CACHE_FOLDER = "cache"
PLAIN_TEXT_FOLDER = "plain_text"
ORIGINAL_TEXT_FOLDER = "original_text"

In [4]:
def load_content(page, params=None):
  if not os.path.exists(CACHE_FOLDER):
    os.makedirs(CACHE_FOLDER)

  #load content
  content = ""
  cache_location = os.path.join(CACHE_FOLDER,page.split('/')[-1].replace('/', '_'))
  if params is not None:
    params_hash = hash(frozenset(params.items()))
    cache_location += "_" + str(params_hash)
  if not os.path.exists(cache_location):
    result = requests.get(page, allow_redirects=True, params=params)
    content = result.text
    with open(cache_location, 'w') as f:
      f.write(content)
  else:
    with open(cache_location) as f:
      content = f.read()
  return content

In [5]:
def slugify(value, allow_unicode=False):
    """
    Taken from https://github.com/django/django/blob/master/django/utils/text.py
    Convert to ASCII if 'allow_unicode' is False. Convert spaces or repeated
    dashes to single dashes. Remove characters that aren't alphanumerics,
    underscores, or hyphens. Convert to lowercase. Also strip leading and
    trailing whitespace, dashes, and underscores.
    """
    value = str(value)
    if allow_unicode:
        value = unicodedata.normalize('NFKC', value)
    else:
        value = unicodedata.normalize('NFKD', value).encode('ascii', 'ignore').decode('ascii')
    value = re.sub(r'[^\w\s-]', '', value.lower())
    return re.sub(r'[-\s]+', '-', value).strip('-_')

#Scrape all articles

In [6]:
def get_all_links():
  df = pd.read_csv('../leichte_sprache_links.csv', header=None)
  all_links = df[:][0]

  leichte_sprache_links = set()
  leichte_sprache_links.update(all_links)

  return list(leichte_sprache_links)

all_links = get_all_links()

In [36]:
titles = [slugify(sub.split('/')[-2].replace('.html','')) for sub in all_links]

link_dataframe = pd.DataFrame(data={'link':all_links,'title':titles})
link_dataframe.to_csv('meta.csv', index=False)

In [37]:
#skip previous cells if you allready scraped the links
link_dataframe = pd.read_csv('meta.csv')

In [38]:
#Define specific processing
LINK_ORIGINAL = re.compile(r"Link zum Original-?KURIER-?Artikel",re.IGNORECASE)
ORIGINAL = re.compile(r"Original-?KURIER-?Artikel",re.IGNORECASE)
DUPLICATED_NEW_LINES = re.compile(r"(?<=\n\n)\s*\n")
DUPLICATED_SPACES = re.compile(r" +")

REMOVE = {
  LINK_ORIGINAL:'',
  ORIGINAL:'',
  DUPLICATED_NEW_LINES:'',
  DUPLICATED_SPACES:' ',
}

def process_text(text):

  for remove_item,replacement in REMOVE.items():
    text = remove_item.sub(replacement, text)

  return text.strip()
  
def get_original_link(text):

  for remove_item,replacement in REMOVE.items():
    text = remove_item.sub(replacement, text)

  return text.strip()

In [39]:
def to_plain_text(soup):
  to_remove = []
  to_remove.extend(soup.find_all('h1'))
  to_remove.extend(soup.find_all('h2'))
  to_remove.extend(soup.find_all('strong'))
  for remove_tag in to_remove:
    if remove_tag is not None:
      remove_tag.decompose()

  paragraphs = soup.find_all('paragraph',attrs={'class':"ng-star-inserted"})
  try:
    plain_text = []
    for paragraph in paragraphs:
        plain_text.append(paragraph.get_text(separator=" ").replace('\n',''))
    plain_text = "\n".join(plain_text)
    plain_text = process_text(plain_text)
    return plain_text.strip()
  except:
    print("Not a valid article")
    return None

def scrape_article(html, scrape_parallel, scrape_monolingual):
  soup = BeautifulSoup(html)
  article_header_title = soup.find("h1", {'class': 'article-header-title'})
  if article_header_title is not None:
    title = article_header_title.get_text().strip()
  else:
    title = None

  if scrape_parallel:
    original_link = soup.find("a", string="Link zum Original-KURIER-Artikel")
  else:
    original_link = None
    
  if scrape_monolingual:
    plain_text = to_plain_text(soup)
  else:
    plain_text = None

  if (original_link is not None):
    print('scraping original article ',original_link["href"])
    try:
        original_html = load_content(original_link["href"])
        soup = BeautifulSoup(original_html)
        original_text = to_plain_text(soup)
    except:
        print("Connection not established")
        original_text = None
        
  else:
    original_text = None

  return plain_text, title, original_text

In [42]:
def load_and_save_plain_text(link, name=None, scrape_parallel = True, scrape_monolingual=True):
  content = load_content(link)

  if name is None:
    name = link.split('/')[-1]

  name = name.replace('/','_') + '.txt'

  #process article
  plain_text, title, original_text = scrape_article(content, scrape_parallel, scrape_monolingual)

  if scrape_monolingual:
    if not os.path.exists(PLAIN_TEXT_FOLDER):
      os.makedirs(PLAIN_TEXT_FOLDER)
    with open(os.path.join(PLAIN_TEXT_FOLDER , name), 'w+') as f:
      f.write(plain_text)
   
  if original_text is not None:
        if not os.path.exists(ORIGINAL_TEXT_FOLDER):
            os.makedirs(ORIGINAL_TEXT_FOLDER)
        with open(os.path.join(ORIGINAL_TEXT_FOLDER , name), 'w+') as f:
            f.write(original_text)

  return {'genre': 'news'}

In [43]:
#load all all links

total_length = len(link_dataframe.index)
for index, row in link_dataframe.iterrows():
  link = row['link']
  title = row['title']
  
  info = load_and_save_plain_text(link, name=title, scrape_monolingual=False)
  if index % 10 == 0:
    print(f"Loaded ({index}/{total_length})")

  #add additional information to meta file
  for key, value in info.items():
    link_dataframe.at[index,key] = value

Loaded (0/4543)
scraping original article  https://kurier.at/leben/projekt-handschlag-so-gewinnt-man-schon-volksschueler-fuer-das-handwert/313.373.119
scraping original article  https://kurier.at/politik/inland/schelling-plaene-reformprogramm-mit-neuem-sparpaket/241.302.188
scraping original article  https://kurier.at/chronik/wien/fruehe-hilfe-fuer-familien-spart-dem-staat-bares-geld/400609142
scraping original article  https://kurier.at/politik/inland/donnerstags-wird-wieder-demonstriert/400136390
scraping original article  https://kurier.at/politik/inland/nationalrat-beschliesst-rechtsanspruch-auf-pflegekarenz/400616462
scraping original article  https://kurier.at/chronik/welt/vulkanexplosion-in-stromboli-100-touristen-verliessen-insel/400542722
scraping original article  https://kurier.at/politik/ausland/drei-stunden-zu-fuss-in-die-arbeit-wie-der-oeffi-streik-paris-laehmt/400704126
scraping original article  https://kurier.at/politik/inland/ministerrat-beschleunigte-bauprojekte-mess

scraping original article  https://kurier.at/genuss/greenpeace-test-ernuechterndes-ergebnis-fuer-apfel-und-orangensaft/400522147
scraping original article  https://kurier.at/wissen/gesundheit/24-stunden-betreuung-so-helfen-die-stillen-heldinnen/400012119
scraping original article  https://kurier.at/lebensart/gesundheit/aerzte-warnen-wir-haben-einen-diabetes-tsunami/191.202.440
scraping original article  https://kurier.at/chronik/niederoesterreich/euratsfeld-vereinsgruendung-aus-spass-am-sport-im-rollstuhl/400675958
scraping original article  https://kurier.at/chronik/oesterreich/burgenland-illegaler-transport-von-46-hundewelpen-gestoppt/308.637.958
scraping original article  https://kurier.at/wissen/ukraine-maedchen-mit-dna-von-drei-eltern-geboren/241.883.508
Loaded (100/4543)
scraping original article  https://kurier.at/chronik/wien/kurier-leser-stellen-flexity-bim-gutes-zeugnis-aus/400340962
scraping original article  https://kurier.at/freizeit/leben-liebe-sex/wiener-psychiater-zum-c

scraping original article  https://kurier.at/wissen/bewegung-in-der-schule-wir-haben-kaum-noch-uebergewichtige/400138202
scraping original article  https://kurier.at/wirtschaft/mehr-perspektiven-fuer-langzeitarbeitslose/400601705
scraping original article  https://kurier.at/chronik/wien/im-kopf-eines-einbrechers/299.160.955
scraping original article  https://kurier.at/leben/dieser-mann-besitzt-die-weltweit-groesste-hello-kitty-sammlung/273.115.678
scraping original article  https://kurier.at/kultur/falco-die-legende-lebt-und-wird-im-jaenner-gefeiert/237.329.635
Loaded (180/4543)
scraping original article  https://kurier.at/genuss/klimawandel-auf-welche-pflanzen-heimische-bauern-jetzt-setzen-muessen/400548029
scraping original article  https://kurier.at/politik/ausland/amtsenthebungsverfahren-trump-dank-republikanern-freigesprochen/400746642
scraping original article  https://kurier.at/politik/ausland/britische-ermittler-nowitschok-taeter-identifiziert/400068533
scraping original articl

scraping original article  https://kurier.at/wirtschaft/alle-fluege-gestrichen-britischer-reisekonzern-thomas-cook-stellt-geschaeft-ein/400613039
scraping original article  https://kurier.at/chronik/oberoesterreich/sich-in-die-rolle-des-anderen-versetzen/400544318
scraping original article  https://kurier.at/wissen/zu-weiche-matratzen-warnung-vor-erstickungsgefahr-fuer-babys/400128803
scraping original article  https://kurier.at/politik/inland/pflege-wird-teurer-aber-wer-zahlts/400146539
scraping original article  https://kurier.at/sport/fussball/fussball-alessandro-schoepf-ist-der-gewinner-des-malta-testspiels/202.055.855
scraping original article  https://kurier.at/sport/fussball/oesterreich-erkaempft-punkt-gegen-wales/224.577.422
scraping original article  https://kurier.at/wissen/weltkatzentag-damit-die-lieblinge-entspannt-schnurren/279.091.136
scraping original article  https://kurier.at/chronik/welt/vergangene-vier-jahre-waren-die-waermsten-der-messgeschichte/400400138
scraping o

scraping original article  https://kurier.at/wirtschaft/coronavirus-aua-setzen-fluege-nach-china-bis-28-februar-aus/400744296
scraping original article  https://kurier.at/leben/neue-alte-masche-maenner-stricken-mit/400640435
scraping original article  https://kurier.at/leben/digitale-lustkiller-macht-das-internet-unser-sexleben-kaputt/400487746
Loaded (350/4543)
scraping original article  https://kurier.at/gesund/faktencheck-machen-uns-klimaanlagen-wirklich-krank/400541087
scraping original article  https://kurier.at/chronik/burgenland/burgenland-foerderungen-fuer-klimaschutz-durch-gemeinden/400726842
scraping original article  https://futurezone.at/digital-life/online-shopping-betrug-vermeiden-die-fallen-bei-amazon-und-co/400107446
scraping original article  https://kurier.at/wirtschaft/karriere/mitarbeiter-halten-nicht-vergraulen/307.351.036
scraping original article  https://kurier.at/chronik/welt/friedensnobelpreis-bekanntgabe-der-gewinner/400136927
scraping original article  https

...

scraping original article  https://kurier.at/chronik/250-jahre-prater-fuer-immer-jung/215.397.778
scraping original article  https://kurier.at/wirtschaft/erste-millionenstrafe-gegen-geplanten-verschleiss/400154985
scraping original article  https://kurier.at/chronik/wien/teile-von-wiener-ring-und-operngasse-wegen-baustellen-gesperrt/400576637
scraping original article  https://kurier.at/politik/ausland/us-wahl/trump-kann-nur-gewinnen/229.857.785
Loaded (4150/4543)
scraping original article  https://kurier.at/wirtschaft/zu-wenig-arbeitskraefte-usa-holen-15-000-auslaender/275.645.150
scraping original article  https://kurier.at/politik/ausland/italiens-innenminister-salvini-setzt-asylwerber-vor-die-tuer/400386503
scraping original article  https://kurier.at/genuss/fluechtlinge-kochen-wo-man-bei-fattoush-ins-gespraech-kommt/400550141
scraping original article  https://kurier.at/wissen/zahl-der-berggorillas-ist-deutlich-gestiegen/400044020
scraping original article  https://kurier.at/chron

scraping original article  https://kurier.at/politik/inland/kira-gruenberg-ist-neue-im-kurz-team/277.500.494
Loaded (4230/4543)
scraping original article  https://kurier.at/chronik/oesterreich/tempo-140-ist-mit-1-maerz-wieder-geschichte/400749528
scraping original article  https://kurier.at/wirtschaft/nachhilfe-fuer-schueler-stunde-kostet-bis-zu-60-euro/400493461
scraping original article  https://kurier.at/wirtschaft/wifo-oesterreichs-wirtschaft-waechst-ueberdurchschnittlich-schnell/279.626.405
scraping original article  https://kurier.at/wirtschaft/air-berlin-ist-finanziell-am-boden/280.762.866
scraping original article  https://kurier.at/wissen/auch-ratten-sind-kitzelig-und-lachen/230.025.804
scraping original article  https://kurier.at/freizeit/leben-liebe-sex/oesterreicher-blicken-optimistisch-in-die-zukunft/400712709
scraping original article  https://kurier.at/gesund/aeltester-mann-der-welt-mit-113-jahren-gestorben/400383332
Loaded (4240/4543)
scraping original article  https://

scraping original article  https://kurier.at/chronik/niederoesterreich/buergermeister-lehrer-und-eltern-wollen-eigene-direktorin/221.544.043
scraping original article  https://kurier.at/chronik/welt/beben-auf-lombok-ueber-500-wanderer-sitzen-auf-vulkan-fest/400075244
scraping original article  https://kurier.at/chronik/welt/300-millionen-menschen-von-hochwasser-bedroht/400661318
Loaded (4320/4543)
scraping original article  https://kurier.at/chronik/wien/tierschuetzer-pelz-verbot-auf-wiener-christkindlmaerkten-ignoriert/400349194
scraping original article  https://kurier.at/politik/ausland/brexit-abstimmung-im-unterhaus-in-london-am-15-jaenner/400370246
scraping original article  https://kurier.at/politik/inland/kopftuch-das-verbot-dreht-sich-nicht-um-die-religion/400016770
scraping original article  https://kurier.at/politik/inland/emmanuel-macron-kommt-nach-salzburg/277.714.368
scraping original article  https://kurier.at/chronik/welt/toedliche-angriffe-auf-moscheen-erschuettern-neus

scraping original article  https://kurier.at/chronik/niederoesterreich/neue-radar-technologie-gegen-wildunfaelle/400601822
scraping original article  https://kurier.at/wissen/wo-forscher-jetzt-die-laengste-salzhoehle-der-welt-entdeckt-haben/400450093
scraping original article  https://kurier.at/chronik/welt/gott-steh-mir-bei-hunderte-tote-bei-tsunami-in-indonesien/400131773
scraping original article  https://kurier.at/wirtschaft/karriere/zwinker-rose-daumen-hoch-emojis-in-der-arbeitswelt/400427387
Loaded (4410/4543)
scraping original article  https://kurier.at/politik/ausland/libyen-eu-will-dramatische-situation-in-fluechtlingscamps-verbessern/245.205.347
scraping original article  https://kurier.at/politik/ausland/stop-soros-ungarn-will-hilfe-fuer-illegale-fluechtlinge-kriminalisieren/400043068
scraping original article  https://kurier.at/chronik/oesterreich/karawankentunnel-voraussichtlich-am-30-juli-gesperrt/211.224.627
scraping original article  https://kurier.at/chronik/weltchroni

scraping original article  https://kurier.at/genuss/food-trends-sommer-2017-slushie-switchel-und-swineapple/266.899.165
scraping original article  https://kurier.at/chronik/wien/oesterreichs-erste-first-lady-martha-kyrle-gestorben/275.921.185
scraping original article  https://kurier.at/sport/fussball/gegen-tel-aviv-gespielt-iran-sperrt-zwei-teamspieler/279.826.736
scraping original article  https://kurier.at/politik/inland/nationalrat-live-12-stunden-tag-und-fremdenrechts-verschaerfung/400061651
scraping original article  https://kurier.at/sport/usain-bolt-startet-seine-fussballkarriere-in-australien/400067483
scraping original article  https://kurier.at/leben/sandmann-bekommt%20gebaerdendolmetscher/228.605.007
Loaded (4490/4543)
scraping original article  https://kurier.at/politik/inland/kopftuchverbot-ziffernnoten-co-das-bringt-das-neue-schuljahr/400582073
scraping original article  https://kurier.at/kultur/naturhistorisches-museum-wien-online-begehbar/220.938.864#
scraping original

#Save scraped data

In [44]:
!zip -r /content/original_corpus.zip /content/original_text -j
# link_dataframe.to_csv('meta.csv', index=False)

	zip warning: name not matched: /content/original_text

zip error: Nothing to do! (try: zip -r -j /content/original_corpus.zip . -i /content/original_text)
